### OSHA Scraper Workbook

### 1. Statement

This workbook is an illustration on extracting, formulating, and exporting companies' data from OSHA database. The following codes are only for internal use of ABB's consulting practicum project and research purpose. For more details of the data source, please refer to https://www.osha.gov/ords/imis/establishment.html.

### 2. Code

In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlencode
import urllib.parse
from copy import copy

#### 2.1 Scrape index

##### 2.1.1 *get_osha_url(query, start_date_str, end_date_str)*

The `get_sha_url()` functin returns the violation information given the company names, start date, and end date. 

In [5]:
def get_osha_url(query, start_date_str, end_date_str):

    results_per_page = 1000
    start_date = pd.to_datetime(start_date_str)
    end_date = pd.to_datetime(end_date_str)

    # make params dictionary
    base_url = f"https://www.osha.gov/ords/imis/establishment.search"
    params = {
        'establishment': '+'.join(query.split()),
        'state': "all",
        'officetype': "all",
        'office': "all",
        'sitezip': 100000,
        'startmonth': start_date.month,
        'startday': start_date.day,
        'startyear': start_date.year,
        'endmonth': end_date.month,
        'endday': end_date.day,
        'endyear': end_date.year,
        'p_case': "all",
        'p_sort': 12,
        'p_desc': "DESC",
        'p_direction': "Prev",
        'p_show': results_per_page,
        'p_violations_exist': "yes"
    }

    return base_url + '?' + urlencode(params)

##### 2.1.2 *get_index_table(osha_url)*

The `get_index_table()` fetch the tables within the results given the osha url.

In [6]:
def get_index_table(osha_url):
    tables = pd.read_html(osha_url)
    if len(tables) >= 3:
      return tables[2]
    # Continue with your DataFrame processing
    else:
      return("Insufficient tables in the HTML content.")

In [11]:
df = pd.read_excel("master_excel_all_variables.xlsx", sheet_name='master_excel_all_variables')
companies = df['company_name_OSHA'].tolist()
companies = [entry.replace('\r', '') if isinstance(entry, str) else entry for entry in companies]
companies = [entry for entry in companies if not (isinstance(entry, float))]

In [12]:
# Specify the start date and end date
date_start = '2018-11-18'
date_end = '2023-11-18'

The following queries are the inputs of get_osha_url() function.

In [13]:
queries = [(company, date_start, date_end) for company in companies]
queries

[('Archer Daniels Midland', '2018-11-18', '2023-11-18'),
 ('American Crystal Sugar', '2018-11-18', '2023-11-18'),
 ('Anheuser Busch', '2018-11-18', '2023-11-18'),
 ('B&G Foods', '2018-11-18', '2023-11-18'),
 ('Blue Bell Creameries, L. P', '2018-11-18', '2023-11-18'),
 ('Brakebush Brothers, Inc.', '2018-11-18', '2023-11-18'),
 ('Brown-Forman', '2018-11-18', '2023-11-18'),
 ('Bunge North America', '2018-11-18', '2023-11-18'),
 ('California Dairies Inc\n', '2018-11-18', '2023-11-18'),
 ('Campbell Soup', '2018-11-18', '2023-11-18'),
 ('Cargill, Incorporated', '2018-11-18', '2023-11-18'),
 ('Cheese Merchants Of America Llc', '2018-11-18', '2023-11-18'),
 ('Chobani, Llc\n', '2018-11-18', '2023-11-18'),
 ('Colgate Palmolive Company', '2018-11-18', '2023-11-18'),
 ('Conagra ', '2018-11-18', '2023-11-18'),
 ('Corteva Agriscience', '2018-11-18', '2023-11-18'),
 ('Country Fresh, Llc', '2018-11-18', '2023-11-18'),
 ('Darigold Inc', '2018-11-18', '2023-11-18'),
 ('Dsm Nutritional Products', '2018-1

##### 2.1.3 *get_activities_df(query)*

The get_activities_df() function uses queries as its only argument, and returns a list of records of violations of a specific companies. By using a for loop, we can combine the companies' violation information in the `activities_df` table.

In [14]:
activities_df = []

def get_activities_df(query):
    establishment, start, end = query
    osha_url = get_osha_url(establishment, start, end)

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        response = requests.get(osha_url, headers=headers)
        tables = pd.read_html(response.content)

        if not tables:
            print(f"No tables found for {query}")
            return None

        index_df_for_this_query = tables[1]
        index_df_for_this_query = index_df_for_this_query.drop(columns=['Unnamed: 0', '#'])

        index_df_for_this_query['query'] = establishment
        index_df_for_this_query['query_start'] = start
        index_df_for_this_query['query_end'] = end

        print(f"Found {len(index_df_for_this_query)} results for {query}\n")
        return index_df_for_this_query

    except Exception as e:
        print(f"Error processing {query}: {e}")
        return None
    
for query in queries:
    activities_df_for_query = get_activities_df(query)

    if activities_df_for_query is not None:
        activities_df.append(activities_df_for_query)

activities_df = pd.concat(activities_df)
# activities_df.to_csv('data/activities_df.csv', index=False)
activities_df

Found 12 results for ('Archer Daniels Midland', '2018-11-18', '2023-11-18')

Found 4 results for ('American Crystal Sugar', '2018-11-18', '2023-11-18')

Found 7 results for ('Anheuser Busch', '2018-11-18', '2023-11-18')

Found 3 results for ('B&G Foods', '2018-11-18', '2023-11-18')

Found 3 results for ('Blue Bell Creameries, L. P', '2018-11-18', '2023-11-18')

Found 5 results for ('Brakebush Brothers, Inc.', '2018-11-18', '2023-11-18')

Found 3 results for ('Brown-Forman', '2018-11-18', '2023-11-18')

Found 7 results for ('Bunge North America', '2018-11-18', '2023-11-18')

Found 2 results for ('California Dairies Inc\n', '2018-11-18', '2023-11-18')

Found 2 results for ('Campbell Soup', '2018-11-18', '2023-11-18')

Found 4 results for ('Cargill, Incorporated', '2018-11-18', '2023-11-18')

Found 2 results for ('Cheese Merchants Of America Llc', '2018-11-18', '2023-11-18')

Found 2 results for ('Chobani, Llc\n', '2018-11-18', '2023-11-18')

Error processing ('Colgate Palmolive Company',

,Activity,Date Opened,RID,ST,Type,Scope,SIC,NAICS,Violations,Establishment Name,query,query_start,query_end
0,1664999.015,04/21/2023,524500,IL,Fat/Cat,Partial,NaN,311224,4,Archer-Daniels-Midland Company,Archer Daniels Midland,2018-11-18,2023-11-18
1,1662897.015,04/12/2023,524500,IL,Fat/Cat,Partial,NaN,311224,1,Archer-Daniels-Midland Company,Archer Daniels Midland,2018-11-18,2023-11-18
2,1659430.015,03/24/2023,452110,KY,Complaint,Partial,2899.0,325998,1,65986 - Archer Daniels Midland Co,Archer Daniels Midland,2018-11-18,2023-11-18
3,1643826.015,01/13/2023,524500,IL,Referral,Partial,NaN,311221,1,Archer-Daniels-Midland Company,Archer Daniels Midland,2018-11-18,2023-11-18
4,1641641.015,01/04/2023,728900,NE,Referral,Partial,NaN,311224,4,Archer Daniels Midland Company,Archer Daniels Midland,2018-11-18,2023-11-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,1383619.015,03/07/2019,729300,MO,Referral,Partial,NaN,311615,3,"Tyson Foods, Inc.","Tyson Foods, Inc.",2018-11-18,2023-11-18
17,1377139.015,02/07/2019,626300,TX,Complaint,Partial,NaN,311615,1,"Tyson Foods, Inc.","Tyson Foods, Inc.",2018-11-18,2023-11-18
0,1472257.015,04/13/2020,728500,MO,Fat/Cat,Partial,NaN,311919,1,Unilever,Unilever,2018-11-18,2023-11-18
1,1386411.015,03/20/2019,521700,IL,Referral,Partial,NaN,311941,1,"Unilever Illinois Manufacturing, Llc",Unilever,2018-11-18,2023-11-18


#### 2.2 Scrape individual activities pages

After getting the records of violation activities for each company, we now further explore the details in each individual activity. For example, the `violation items`, `violation summary`, and other `related activities` could also be found in this table. Therefore, we need several functions to extract the corresponding details of each activity.

In [15]:
# Figure out how to scrape individual activity
def get_related_activity_table(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    tables = pd.read_html(response.content, match="Related Activity")
    return tables[0]

def get_violation_summary_table(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    tables= pd.read_html(response.content,match="Violation Summary")

    return tables[0]

def get_violation_items_table(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    tables= pd.read_html(response.content,match="Violation Items")

    return tables[0]

def get_investigated_inspection_table(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    tables = pd.read_html(response.content, match="Investigated Inspection")
    return tables[0]

def extract_key_value(tag):
    assert ':' in tag.text
    assert tag.find('strong')

    key = tag.text.split(':')[0].strip()
    value = tag.text.split(':')[1].strip()
    return key, value

def get_main_container(inspection_url):
    # get HTML from inspection page
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(inspection_url,headers=headers)
    html_doc = response.text
    soup = BeautifulSoup(html_doc, 'html.parser')

    # get main container (ignore footer/header/etc...)
    html_main_container = soup.find(id="maincontain")

    return html_main_container

def has_investigation_summary(html_main_container):
    h4s = html_main_container.find_all('h4')
    for h4 in h4s:
        if h4.text.strip() == "Investigation Summary":
            return True
    return False

def get_details(html_main_container):

    # Delete anything after "Investigation Summary H4"
    # we will scrape that stuff in a separate function
    h4s = html_main_container.find_all('h4')
    for h4 in h4s:
        if h4.text == "Investigation Summary":
            for e in h4.find_all_next():
                e.clear()

    # get details
    details = {}

    # get case status
    html_wells = html_main_container.find_all("div", class_="well")
    try:
        assert len(html_wells) == 2
        assert html_wells[0] == html_wells[1]
        case_status = html_wells[0]
        key, value = extract_key_value(case_status)
    except:
        key = 'Case Status'
        value = 'ERROR'
        print("ERROR - couldn't scrape case status")
    details[key] = value

    # get remaining details
    html_spans = html_main_container.find_all("div", class_="span4")
    for span in html_spans:
        html_p_tags = span.find_all('p')
        if len(html_p_tags) == 0:
            columns_without_colon = 0
            key, value = extract_key_value(span)
            details[key] = value
        else:
            for p_tag in html_p_tags:
                key, value = extract_key_value(p_tag)
                details[key] = value

    return details

def get_investigation_summary_details(html_main_container):
    # Delete anything after "Investigation Summary H4"
    # we will scrape that stuff in a separate function
    h4s = copy(html_main_container).find_all('h4')
    for h4 in h4s:
        if h4.text == "Investigation Summary":
            for e in h4.find_previous_siblings():
                e.decompose()

    details = {}

    spans = html_main_container.find_all('div', class_='span4')
    p_tags = html_main_container.find_all('p')
    spans_and_ptags = spans + p_tags

    details = {}
    notes_columns = 0
    for tag in spans_and_ptags:
        if ':' in tag.text and tag.find('strong'):
            key, value = extract_key_value(tag)
        else:
            notes_columns += 1
            key = f"investigation_summary_notes_{notes_columns}"
            value = tag.text
        details[key] = value

    return(details)

Then we create different lists to store the details.

In [16]:
violation_summary_tables = []
violation_items_tables = []
related_activity_tables = []
details_dictionaries = []
investigated_inspections_table = []

for index, row in activities_df.reset_index(drop=True).iterrows():
    activity_code = row['Activity']
    establishment=row['Establishment Name']
    url = f"https://www.osha.gov/ords/imis/establishment.inspection_detail?id={activity_code}"
    print(f"{index} of {len(activities_df)} - scraping {url}")

    violation_summary = get_violation_summary_table(url)
    violation_summary['activity_code'] = activity_code
    violation_summary['establishment'] = establishment
    violation_summary_tables.append(violation_summary)
    #print(violation_summary)
    violation_items = get_violation_items_table(url)
    violation_items['activity_code'] = activity_code
    violation_items['establishment'] = establishment
    violation_items_tables.append(violation_items)

    try:
        related_activity = get_related_activity_table(url)
        related_activity['activity_code'] = activity_code
        related_activity['establishment'] = establishment
        related_activity_tables.append(related_activity)
    except:
        print("ERROR - related activity table wasn't scraped")


    html = get_main_container(url)
    details = {}

    if has_investigation_summary(html):
        print("Has investigation summary")
        investigation_summary_details = get_investigation_summary_details(html)
        details.update(investigation_summary_details)

        investigated_inspections = get_investigated_inspection_table(url)
        investigated_inspections['activity_code'] = activity_code
        investigated_inspections['establishment'] = establishment
        investigated_inspections_table.append(investigated_inspections)


    details.update(get_details(html))
    details['activity_code'] = activity_code
    details['establishment'] = establishment
    details_dictionaries.append(details)

0 of 269 - scraping https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1664999.015
1 of 269 - scraping https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1662897.015
Has investigation summary
2 of 269 - scraping https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1659430.015
3 of 269 - scraping https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1643826.015
4 of 269 - scraping https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1641641.015
5 of 269 - scraping https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1627196.015
6 of 269 - scraping https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1610433.015
ERROR - related activity table wasn't scraped
7 of 269 - scraping https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1587740.015
8 of 269 - scraping https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1583701.015
Has investigation summary
9 of 269 - scraping 

In [17]:
violation_summary_df = pd.concat(violation_summary_tables)
violation_items_df = pd.concat(violation_items_tables)
related_activity_df = pd.concat(related_activity_tables)
investigated_inspections_df = pd.concat(investigated_inspections_table)
details_df = pd.DataFrame(details_dictionaries)

Specifically, we only focus a 6 categories of violations sorted by the `Standard Cited` column of violation_items table. The encoding of each category of violations could be viewed on https://www.osha.gov/laws-regs/federalregister/standardnumber/1910. Therefore, we map the category name into the violation_items_df table.

In [18]:
mapping_dict = {
    '19100269': 'Electric Power Generation, Transmission, and Distribution',
    '19100333': 'Selection and use of work practices',
    '19100334': 'Use of equipment',
    '19100335': 'Safeguards for personnel protection',
    '19100137': 'Electrical Protective Equipment',
    '19100147': 'The control of hazardous energy (lockout/tagout)'
}

In [19]:
filtered_violation_items_df = violation_items_df[violation_items_df['Standard Cited'].str.startswith(tuple(mapping_dict.keys()))]
filtered_violation_items_df['Citation Category'] = filtered_violation_items_df['Standard Cited'].apply(lambda x: mapping_dict.get(x.split()[0], None))
filtered_violation_items_df = filtered_violation_items_df[['Citation ID', 'Standard Cited', 'Issuance Date','activity_code', 'establishment','Citation Category']]
filtered_violation_items_df

C:\Users\xben4\AppData\Local\Temp\ipykernel_33008\226249089.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_violation_items_df['Citation Category'] = filtered_violation_items_df['Standard Cited'].apply(lambda x: mapping_dict.get(x.split()[0], None))


,Citation ID,Standard Cited,Issuance Date,activity_code,establishment,Citation Category
0,1001,19100147 C04 I,03/20/2023,1627196.015,Archer Daniels Midland,The control of hazardous energy (lockout/tagout)
1,1002,19100147 D06,03/20/2023,1627196.015,Archer Daniels Midland,The control of hazardous energy (lockout/tagout)
0,01001A,19100147 C04 I,06/22/2021,1513727.015,American Crystal Sugar,The control of hazardous energy (lockout/tagout)
1,01001B,19100147 C07 I,06/22/2021,1513727.015,American Crystal Sugar,The control of hazardous energy (lockout/tagout)
0,01001A,19100147 C04 II B,11/30/2021,1539718.015,"Anheuser-Busch, Llc.",The control of hazardous energy (lockout/tagout)
...,...,...,...,...,...,...
2,01001C,19100147 C06 I,04/25/2022,1572930.015,"Tyson Foods, Inc.",The control of hazardous energy (lockout/tagout)
0,1001,19100147 C07 III B,12/18/2020,1497467.015,The Hillshire Brands Company Dba Tyson Foods Inc.,The control of hazardous energy (lockout/tagout)
2,2001,19100147 C07 I A,04/08/2019,1383619.015,"Tyson Foods, Inc.",The control of hazardous energy (lockout/tagout)
0,1001,19100147 C04 I,06/21/2023,1666264.015,J.M. Smucker Llc,The control of hazardous energy (lockout/tagout)


Furthermore, we also want to explore the summary of that violation activity, therefore, we can merge the filtered_violation_items_df with the details_df by `activity_code` column.

In [20]:
merged_df = pd.merge(filtered_violation_items_df, details_df[['activity_code', 'investigation_summary_notes_1', 'investigation_summary_notes_2', 'investigation_summary_notes_3', 'Keywords']],
                     on='activity_code', how='inner')

In [21]:
merged_df

,Citation ID,Standard Cited,Issuance Date,activity_code,establishment,Citation Category,investigation_summary_notes_1,investigation_summary_notes_2,investigation_summary_notes_3,Keywords
0,1001,19100147 C04 I,03/20/2023,1627196.015,Archer Daniels Midland,The control of hazardous energy (lockout/tagout),NaN,NaN,NaN,NaN
1,1002,19100147 D06,03/20/2023,1627196.015,Archer Daniels Midland,The control of hazardous energy (lockout/tagout),NaN,NaN,NaN,NaN
2,01001A,19100147 C04 I,06/22/2021,1513727.015,American Crystal Sugar,The control of hazardous energy (lockout/tagout),NaN,NaN,NaN,NaN
3,01001B,19100147 C07 I,06/22/2021,1513727.015,American Crystal Sugar,The control of hazardous energy (lockout/tagout),NaN,NaN,NaN,NaN
4,01001A,19100147 C04 II B,11/30/2021,1539718.015,"Anheuser-Busch, Llc.",The control of hazardous energy (lockout/tagout),NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
128,01001C,19100147 C06 I,04/25/2022,1572930.015,"Tyson Foods, Inc.",The control of hazardous energy (lockout/tagout),Employee Amputates Finger In Chain And Sprocket,,"At 11:00 a.m. on January 6, 2022, an employee ...","amputated, amputation, chain, conveyor, finger..."
129,1001,19100147 C07 III B,12/18/2020,1497467.015,The Hillshire Brands Company Dba Tyson Foods Inc.,The control of hazardous energy (lockout/tagout),NaN,NaN,NaN,NaN
130,2001,19100147 C07 I A,04/08/2019,1383619.015,"Tyson Foods, Inc.",The control of hazardous energy (lockout/tagout),Employee Amputates Finger When Places Hand In ...,,"At approximately 10:15 p.m. on March 5, 2019,...","amputated, amputation, cleaning, finger, finge..."
131,1001,19100147 C04 I,06/21/2023,1666264.015,J.M. Smucker Llc,The control of hazardous energy (lockout/tagout),NaN,NaN,NaN,NaN


In [ ]:
merged_df.to_excel('OSHA_violations.xlsx', index=False)